In [29]:
import lyricsgenius as lg
import pandas as pd
import numpy as np
import re
import json
import time
import warnings
import requests
import text2emotion as te
warnings.simplefilter(action='ignore', category=FutureWarning)
GENIUS_TOKEN = "Y80WiHQGGXOuJZ1CHjuO1U5IUGnW6I1ftwCxCFBTB88VZHYL8XxBewoBibWrh05O"
APISONGKEY = "dd802c9d07fe0dca4580447eb899dbd2"
genius = lg.Genius(GENIUS_TOKEN)
lyricsDF = pd.DataFrame(columns=["Album", "Artist","Song #","Song Name", "Lyrics"])
reviewsDF = pd.read_csv('./fantano_reviews.csv', index_col=0)
reviewsDF.sort_values(by=['Artist', 'Album'], inplace=True)

# Helper Functions

In [4]:
# Fix the unicode encoding for the artist and album names
def unicodeEncode(text):
    return str(text).encode('utf-8', 'ignore').decode('utf-8')

# Remove the song name from the lyrics
def removeLyrics(song_name, lyrics):
    song_name,lyrics = str(song_name) + " Lyrics",str(lyrics)
    if song_name in lyrics:
        lyrics = lyrics.replace(song_name,"")
    return lyrics

def text2Emotions(text):
    emotions = te.get_emotion(str(text))
    return emotions["Angry"], emotions["Fear"], emotions["Happy"], emotions["Sad"], emotions["Surprise"]

def getVocab(corpus):
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    vocab = vectorizer.get_feature_names()
    
    # lets just get rid of any short words
    return [word for word in vocab if len(word) > 2 and not word.isdigit()]
    
def getRhymes(word):
    
    url = f"https://api.datamuse.com/words?rel_rhy={word.lower()}"
    
    response = requests.get(url)
    
    return response.json()

# Get Lyrics of each song (Takes 18+ hours)

In [4]:
# Get lyrics for each song in each album
for index,value in reviewsDF["Album"].items():
    album = str(value).strip()
    artist = reviewsDF["Artist"][index].strip()
    song_count = 0
    try:
        time.sleep(6)
        songs = genius.search_album(album,artist)
        for track in songs.tracks:
            song_count += 1
            count = str(song_count) + "/" + str(len(songs.tracks))
            song_name = track.song.title
            lyrics = re.sub(r"\[.+\]\n", ' ', track.to_text()) # Remove [Verse 1], [Chorus], etc.
            if lyrics.endswith("You might also like"): # Remove "You might also like" at the end
                lyrics = lyrics[:-19]
            lyrics = lyrics.strip().replace("\n"," ")
            row = {"Album": album, "Artist": artist, "Song #": count, "Song Name": song_name, "Lyrics": lyrics}
            lyricsDF = lyricsDF.append(row, ignore_index=True)
    except Exception as e:
        print(e)
        pass
    print(f"Total Songs: {lyricsDF.shape[0]}")
lyricsDF.to_csv('./fantano_lyrics.csv')

Searching for "mandatory fun" by "weird al" yankovic...
Total Songs: 12
Searching for "ethereal" by $not...
Total Songs: 26
Searching for "i want to die in new orleans" by $uicideboy$...
Total Songs: 40
Searching for "long term effects of suffering" by $uicideboy$...
Total Songs: 53
Searching for "house of sugar" by (sandy) alex g...
Total Songs: 67
Searching for "rocket" by (sandy) alex g...
Total Songs: 81
Searching for "modus vivendi" by 070 shake...
Total Songs: 95
Searching for "you can't kill me" by 070 shake...
Total Songs: 109
Searching for "1000 gecs" by 100 gecs...
Total Songs: 119
Searching for "1000 gecs and the tree of clues" by 100 gecs...
Total Songs: 138
Searching for "based on a t.r.u. story" by 2 chainz...
Total Songs: 155
Searching for "rap or go to the league" by 2 chainz...
Total Songs: 169
Searching for "i am > i was" by 21 savage...
Total Songs: 184
Searching for "issa album" by 21 savage...
Total Songs: 198
Searching for "savage mode ii" by 21 savage & metro boo

In [5]:
lyricsDF = pd.read_csv('./fantano_lyrics.csv', index_col=0)
lyricsDF

,Album,Artist,Song #,Song Name,Lyrics
0,mandatory fun,"""weird al"" yankovic",1/12,Handy,"Handy Lyrics First things first, I'm a craftsm..."
1,mandatory fun,"""weird al"" yankovic",2/12,Lame Claim to Fame,Lame Claim to Fame Lyrics One time I was in th...
2,mandatory fun,"""weird al"" yankovic",3/12,Foil,Foil Lyrics I never seem to finish all my food...
3,mandatory fun,"""weird al"" yankovic",4/12,Sports Song,Sports Song Lyrics Your sports team is vastly ...
4,mandatory fun,"""weird al"" yankovic",5/12,Word Crimes,Word Crimes Lyrics Everybody shut up! (Woo!) E...
...,...,...,...,...,...
26922,xe,zs,1/5,The Future of Royalty,NaN
26923,xe,zs,2/5,Wolf Government,NaN
26924,xe,zs,3/5,Corps,NaN
26925,xe,zs,4/5,Weakling,NaN


# Fix lyrics before processing

In [6]:
lyricsDF["Lyrics"] = lyricsDF["Lyrics"].apply(unicodeEncode) #Fix Unicode encoding
lyricsDF["Song Name"] = lyricsDF["Song Name"].apply(unicodeEncode) #Fix Unicode encoding
lyricsDF["Lyrics"] = lyricsDF["Lyrics"].str.replace(r"\[[^\]]*\]", ' ').str.replace(r"\\", '',).str.replace(r"[0-9][0-9]Embed","").str.replace(r"Translations.*Lyrics","") # Remove [Verse 1], [Chorus], etc.
lyricsDF["Lyrics"] = lyricsDF.apply(lambda x: removeLyrics(x["Song Name"], x["Lyrics"]), axis=1) # Remove song name from lyrics

# Get emotions of the lyrics (Takes 27+ Hours)

In [9]:
lyricsDF["Angry"],lyricsDF["Fear"],lyricsDF["Happy"],lyricsDF["Sad"],lyricsDF["Surprise"] = zip(*lyricsDF["Lyrics"].apply(text2Emotions))
lyricsDF.to_csv('./fantano_lyrics.csv')

{'Happy': 0.04, 'Angry': 0.04, 'Surprise': 0.35, 'Sad': 0.33, 'Fear': 0.23}
{'Happy': 0.16, 'Angry': 0.08, 'Surprise': 0.44, 'Sad': 0.2, 'Fear': 0.12}
{'Happy': 0.13, 'Angry': 0.08, 'Surprise': 0.1, 'Sad': 0.23, 'Fear': 0.46}
{'Happy': 0.44, 'Angry': 0.0, 'Surprise': 0.11, 'Sad': 0.15, 'Fear': 0.3}
{'Happy': 0.03, 'Angry': 0.09, 'Surprise': 0.62, 'Sad': 0.18, 'Fear': 0.09}
{'Happy': 0.03, 'Angry': 0.0, 'Surprise': 0.35, 'Sad': 0.12, 'Fear': 0.5}
{'Happy': 0.29, 'Angry': 0.09, 'Surprise': 0.31, 'Sad': 0.15, 'Fear': 0.16}
{'Happy': 0.12, 'Angry': 0.08, 'Surprise': 0.08, 'Sad': 0.25, 'Fear': 0.46}
{'Happy': 0.04, 'Angry': 0.15, 'Surprise': 0.35, 'Sad': 0.2, 'Fear': 0.26}
{'Happy': 0.07, 'Angry': 0.02, 'Surprise': 0.14, 'Sad': 0.14, 'Fear': 0.63}
{'Happy': 0.13, 'Angry': 0.05, 'Surprise': 0.21, 'Sad': 0.37, 'Fear': 0.24}
{'Happy': 0.1, 'Angry': 0.07, 'Surprise': 0.23, 'Sad': 0.17, 'Fear': 0.43}
{'Happy': 0.04, 'Angry': 0.13, 'Surprise': 0.39, 'Sad': 0.39, 'Fear': 0.04}
{'Happy': 0.49, 'Ang

In [12]:
lyricsDF = pd.read_csv('./fantano_lyrics.csv', index_col=0)
lyricsDF["Lyrics"] = lyricsDF["Lyrics"].apply(unicodeEncode) #Fix Unicode encoding
lyricsDF["Song Name"] = lyricsDF["Song Name"].apply(unicodeEncode) #Fix Unicode encoding

# Get the amount of rhymes in lyrics

In [ ]:
# Get the rhyming words for each song to add to lyricsDF
songs_list = lyricsDF["Lyrics"].tolist()
for index,value in reviewsDF["Album"].items():
    rhyme = 0
    lyrics = songs_list[index]
    vocab = get_vocab([lyrics])

    for word in vocab:
        rhymes_response = get_rhymes(word)
        if len(rhymes_response) > 0: # If there are rhymes for the word
            rhyming_words = [rhyme['word'] for rhyme in rhymes_response] + [word]
    